In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import spei as si 
import geopandas as gpd
from shapely.geometry import Point

### Import stations data

In [2]:
# Import data and metadata

# Load Metadata 
metadata_path = "insert path to station data excel file"
meta_df = pd.read_excel(metadata_path)
meta_df.set_index('Station',inplace=True)

# Select the columns that are necessary for this code
meta_df = meta_df.iloc[:, 0:2]

# Load data
df_path = "path to pre-processed stations metadata excel file"
df = pd.read_excel(df_path)
df = df.iloc[:,1:]
df.set_index('date',inplace=True)

### Import boundaries for spatial distribution

In [3]:
# Read the shapefile
shapefile_path = 'Insert path to ecorigions (or other boundaries) shapefile'
areas = gpd.read_file(shapefile_path)

### Compute the SGI for all the stations

In [ ]:
# Create a new DataFrame to store the SGI values

df_sgi = pd.DataFrame()

# Apply the SGI function to each column of the DataFrame

for column in df.columns:
    df_sgi[column] = si.sgi(df[column])

# Reset the index & Transpose the DataFrame
dates = df_sgi.index.to_pydatetime()
df_sgi = df_sgi.reset_index(drop = True)
df_sgi = df_sgi.T


#### Perform the average of the SGI values at each timestep for each polygon in the shapefile

In [ ]:
# Join with coordinate information in metadata dataframe
df_latlon = df_sgi.join(meta_df)

# Convert the stations dataframe into a GeoDataFrame by specifying the geometry column with the coordinates:

geometry = [Point(xy) for xy in zip(df_latlon['Longitude'], df_latlon['Latitude'])]
stations_gdf = gpd.GeoDataFrame(df_latlon, geometry=geometry)

# Perform a spatial join between the polygons and the stations to determine which stations fall within each polygon:
stations_by_polygon = gpd.sjoin(areas, stations_gdf, how='inner', op='contains')

# Select the columns based on their numeric names
time_step_columns = [col for col in stations_by_polygon.columns if str(col).isdigit()]

# Group the stations by polygon and time step, and calculate the average for each group:
averages = stations_by_polygon.groupby(['WMA_NewN'])[time_step_columns].mean()


#### Plot the resulting time series by area

In [ ]:

# Iterate through the rows of the averages DataFrame
for index, row in averages.iterrows():
    # Extract the polygon ID from the index
    polygon_id = index

    # Create a pandas time series for the plot function to work
    time_series = pd.Series(row.values, index=dates)
    
    # Create a plot for the current row
    plt.figure()

    # Plot the average values for each time step
    si.plot.si(time_series)

    # Set plot title and labels
    plt.title(f"Catchment: {polygon_id}")
    plt.xlabel("Time ")
    plt.ylabel("SGI")

    # Save the plots
    plt.savefig(f'{polygon_id}.png')
